# 코드설명
Tex를 받아와서 수식만 뽑아낸 후

중학생 이상의 수준을 요구하는 수식만 해설

_build 폴더 안에 .tex파일로 생긴다!


### 셀 1. 환경 준비 & 모델 로드 (4-bit, CUDA 자동 감지)

In [1]:
import torch, importlib.metadata as im
print("torch file:", torch.__file__)
print("torch version:", torch.__version__)
print("pip sees torch:", im.version("torch"))
print("CUDA available:", torch.cuda.is_available())


torch file: c:\POLO\venv\Lib\site-packages\torch\__init__.py
torch version: 2.5.1+cu121
pip sees torch: 2.5.1+cu121
CUDA available: True


In [3]:
# === 셀 1: 환경 준비 & 모델 로드 ===
import os, torch, sys, platform, json, re, textwrap, subprocess, shutil
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ----- 기본 경로 설정 -----
# INPUT_TEX_PATH = r"C:\POLO\polo-system\models\math\iclr2022_conference.tex"        # <- 분석할 LaTeX 파일 경로
INPUT_TEX_PATH = r"C:\POLO\polo-system\models\math\yolo.tex"        # <- 분석할 LaTeX 파일 경로
OUT_DIR        = "C:/POLO/polo-system/models/math./_build"          # 산출물 폴더 경로
os.makedirs(OUT_DIR, exist_ok=True)

# ----- 모델/토크나이저 설정 -----
MODEL_ID = "Qwen/Qwen2.5-Math-1.5B-Instruct"   # 1.5B 수학 특화 instruct
USE_4BIT = True                                 # 8GB VRAM을 고려한 4-bit 양자화
DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Device selected: {DEVICE}")

# ----- 모델 로드 -----
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

bnb_config = None
if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    )

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
    trust_remote_code=True 
)

# 공통 generate 설정
GEN_KW = dict(
    max_new_tokens=512,
    temperature=0.2,
    top_p=0.9,
    do_sample=True
)

print("Model & tokenizer loaded.")


Python: 3.11.9
PyTorch: 2.5.1+cu121
CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU
Device selected: cuda
Model & tokenizer loaded.


### 셀 2. LaTeX 파서: 수식 추출(인라인/디스플레이/환경) + 라인번호 매핑

In [4]:
# === 셀 2: LaTeX 수식 추출 유틸 ===
from pathlib import Path

assert Path(INPUT_TEX_PATH).exists(), f"입력 TeX 파일을 찾을 수 없습니다: {INPUT_TEX_PATH}"
src = Path(INPUT_TEX_PATH).read_text(encoding="utf-8", errors="ignore")

# 라인 오프셋 인덱스
lines = src.splitlines()
offsets = []
pos = 0
for ln in lines:
    offsets.append(pos)
    pos += len(ln) + 1

def pos_to_line(p:int)->int:
    lo, hi = 0, len(offsets)-1
    while lo <= hi:
        mid = (lo+hi)//2
        if offsets[mid] <= p:
            lo = mid + 1
        else:
            hi = mid - 1
    return hi + 1  # 1-based

def extract_equations(tex:str)->List[Dict]:
    matches = []
    def add(kind, start, end, body, env=""):
        matches.append({
            "kind": kind, "env": env, "start": start, "end": end,
            "line_start": pos_to_line(start), "line_end": pos_to_line(end),
            "body": body.strip()
        })
    # $$ ... $$
    for m in re.finditer(r"\$\$(.+?)\$\$", tex, flags=re.DOTALL):
        add("display($$ $$)", m.start(), m.end(), m.group(1))
    # \[ ... \]
    for m in re.finditer(r"\\\[(.+?)\\\]", tex, flags=re.DOTALL):
        add("display(\\[ \\])", m.start(), m.end(), m.group(1))
    # \( ... \)
    for m in re.finditer(r"\\\((.+?)\\\)", tex, flags=re.DOTALL):
        add("inline(\\( \\))", m.start(), m.end(), m.group(1))
    # inline $...$ (단, $$ 제외)
    for m in re.finditer(r"(?<!\$)\$(?!\$)(.+?)(?<!\$)\$(?!\$)", tex, flags=re.DOTALL):
        add("inline($ $)", m.start(), m.end(), m.group(1))
    # environments
    envs = ["equation","equation*","align","align*","multline","multline*","gather","gather*","flalign","flalign*","eqnarray","eqnarray*","split"]
    for env in envs:
        pattern = rf"\\begin{{{re.escape(env)}}}(.+?)\\end{{{re.escape(env)}}}"
        for m in re.finditer(pattern, tex, flags=re.DOTALL):
            add(f"env", m.start(), m.end(), m.group(1), env=env)
    # 중복(동일 범위) 제거 및 정렬
    uniq = {}
    for it in matches:
        key = (it["start"], it["end"])
        if key not in uniq:
            uniq[key] = it
    out = list(uniq.values())
    out.sort(key=lambda x: x["start"])
    return out

equations_all = extract_equations(src)
print(f"총 수식 개수: {len(equations_all)}")
equations_all[:2]  # 미리보기


총 수식 개수: 66


[{'kind': 'inline($ $)',
  'env': '',
  'start': 1801,
  'end': 1805,
  'line_start': 63,
  'line_end': 63,
  'body': '^*'},
 {'kind': 'inline($ $)',
  'env': '',
  'start': 1822,
  'end': 1833,
  'line_start': 63,
  'line_end': 63,
  'body': '^{* \\dag}'}]

### 셀 3. “중학생 수준 이상” 판별 휴리스틱

> 기본적으로 ∑, ∂, argmax, 분수·제곱근의 중첩, 좌표/박스 지시자 등 조금 복합적인 표기가 있으면 상위 난이도로 분류합니다. 프로젝트에 맞게 규칙을 쉽게 커스터마이즈할 수 있습니다.

In [ ]:
# -*- coding: utf-8 -*-
"""
LaTeX 수식 해설 API (FastAPI)
- uvicorn --reload app:app 로 실행
- GET  /health                         : 상태 체크
- GET  /count/{file_path:path}         : 수식 개수만 세기(간단 확인용)
- POST /count                          : JSON {"path": "C:\\...\\yolo.tex"}
- GET  /math/{file_path:path}          : 파일 경로를 URL path로 넘겨 실행
- POST /math                           : JSON {"path": "C:\\...\\yolo.tex"}

참고 사항
- 콘솔 출력이 바로 보이도록 stdout 라인 버퍼링 + flush=True 적용
- pad==eos 경고 방지를 위해 pad 토큰 추가 + attention_mask 명시
"""

# === 셀 1: 환경 준비 & 모델 로드 ===
import os, torch, sys, json, re, textwrap, datetime
from typing import List, Dict
from pathlib import Path
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# stdout 라인 버퍼링 (print 즉시 표시)
try:
    sys.stdout.reconfigure(line_buffering=True)
except Exception:
    pass

VERSION = "POLO-Math-API v3 (flush+mask+pad)"; print(VERSION, flush=True)

# ----- 기본 경로 설정 -----
INPUT_TEX_PATH = r"C:\\POLO\\polo-system\\models\\math\\yolo.tex"  # 예시 경로
OUT_DIR        = "C:/POLO/polo-system/models/math/_build"
os.makedirs(OUT_DIR, exist_ok=True)

# ----- 모델/토크나이저 설정 -----
MODEL_ID = "Qwen/Qwen2.5-Math-1.5B-Instruct"
USE_4BIT = False
DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Python: {sys.version.split()[0]}", flush=True)
print(f"PyTorch: {torch.__version__}", flush=True)
print(f"CUDA available: {torch.cuda.is_available()}", flush=True)
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}", flush=True)
print(f"Device selected: {DEVICE}", flush=True)

# ----- 모델 로드 -----
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

    # pad_token 경고 완화: pad 없거나 eos와 같으면 [PAD] 추가
    PAD_ADDED = False
    if tokenizer.pad_token_id is None or tokenizer.pad_token_id == tokenizer.eos_token_id:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        PAD_ADDED = True

    bnb_config = None
    if USE_4BIT:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16
        )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
        quantization_config=bnb_config,
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )

    if PAD_ADDED:
        model.resize_token_embeddings(len(tokenizer))

    # 공통 generate 설정
    GEN_KW = dict(
        max_new_tokens=512,
        temperature=0.2,
        top_p=0.9,
        do_sample=True
    )

    print("Model & tokenizer loaded.", flush=True)
except Exception as e:
    tokenizer = None
    model = None
    GEN_KW = {}
    print("[Model Load Error]", e, flush=True)


# === 공용 유틸: 라인 위치 계산 ===
def make_line_offsets(text: str) -> List[int]:
    lines = text.splitlines()
    offsets, pos = [], 0
    for ln in lines:
        offsets.append(pos)
        pos += len(ln) + 1  # '\n'
    return offsets

def build_pos_to_line(offsets: List[int]):
    def pos_to_line(p: int) -> int:
        lo, hi = 0, len(offsets)-1
        while lo <= hi:
            mid = (lo+hi)//2
            if offsets[mid] <= p:
                lo = mid + 1
            else:
                hi = mid - 1
        return hi + 1  # 1-based
    return pos_to_line


# === 셀 2: LaTeX 수식 추출 유틸 ===
def extract_equations(tex: str, pos_to_line) -> List[Dict]:
    matches: List[Dict] = []

    def add(kind, start, end, body, env=""):
        matches.append({
            "kind": kind, "env": env, "start": start, "end": end,
            "line_start": pos_to_line(start), "line_end": pos_to_line(end),
            "body": body.strip()
        })

    # $$ ... $$
    for m in re.finditer(r"\$\$(.+?)\$\$", tex, flags=re.DOTALL):
        add("display($$ $$)", m.start(), m.end(), m.group(1))
    # \[ ... \]
    for m in re.finditer(r"\\\[(.+?)\\\]", tex, flags=re.DOTALL):
        add("display(\\[ \\])", m.start(), m.end(), m.group(1))
    # \( ... \)
    for m in re.finditer(r"\\\((.+?)\\\)", tex, flags=re.DOTALL):
        add("inline(\\( \\))", m.start(), m.end(), m.group(1))
    # inline $...$ (단, $$ 제외)
    for m in re.finditer(r"(?<!\$)\$(?!\$)(.+?)(?<!\$)\$(?!\$)", tex, flags=re.DOTALL):
        add("inline($ $)", m.start(), m.end(), m.group(1))
    # environments
    envs = ["equation","equation*","align","align*","multline","multline*",
            "gather","gather*","flalign","flalign*","eqnarray","eqnarray*","split"]
    for env in envs:
        pattern = rf"\\begin{{{re.escape(env)}}}(.+?)\\end{{{re.escape(env)}}}"
        for m in re.finditer(pattern, tex, flags=re.DOTALL):
            add("env", m.start(), m.end(), m.group(1), env=env)

    # 중복(동일 범위) 제거 및 정렬
    uniq = {}
    for it in matches:
        key = (it["start"], it["end"])
        if key not in uniq:
            uniq[key] = it
    out = list(uniq.values())
    out.sort(key=lambda x: x["start"])
    return out


# === 셀 3: 난이도 휴리스틱 정의 (요청하신 버전 반영) ===
ADV_TOKENS = [
    r"\\sum", r"\\prod", r"\\int", r"\\lim", r"\\nabla", r"\\partial",
    r"\\mathbb", r"\\mathcal", r"\\mathbf", r"\\boldsymbol",
    r"\\argmax", r"\\argmin", r"\\operatorname", r"\\mathrm\{KL\}",
    r"\\mathbb\{E\}", r"\\Pr", r"\\sigma", r"\\mu", r"\\Sigma", r"\\theta",
    r"\\frac\{[^{}]*\{[^{}]*\}[^{}]*\}",  # 중첩 분수
    r"\\hat\{", r"\\tilde\{", r"\\bar\{", r"\\widehat\{", r"\\widetilde\{",
    r"\\sqrt\{[^{}]*\{",                   # 중첩 sqrt
    r"\\left", r"\\right",
    r"\\in", r"\\subset", r"\\forall", r"\\exists",
    r"\\cdot", r"\\times", r"\\otimes",
    r"IoU", r"\\log", r"\\exp",
    r"\\mathbb\{R\}", r"\\mathbb\{N\}", r"\\mathbb\{Z\}",
    r"\\Delta", r"\\delta", r"\\epsilon", r"\\varepsilon",
]
ADV_RE = re.compile("|".join(ADV_TOKENS))

def count_subscripts(expr: str) -> int:
    return len(re.findall(r"_[a-zA-Z0-9{\\]", expr))

def is_advanced(eq: str) -> bool:
    if ADV_RE.search(eq):
        return True
    if len(eq) > 40 and count_subscripts(eq) >= 2:
        return True
    if "\n" in eq and len(eq) > 30:
        return True
    return False


# === 셀 4: 문서 전체 개요 생성 (모델 요약) ===
def take_slices(text: str, head_chars=4000, mid_chars=2000, tail_chars=4000):
    n = len(text)
    head = text[:min(head_chars, n)]
    mid_start = max((n // 2) - (mid_chars // 2), 0)
    mid = text[mid_start: mid_start + min(mid_chars, n)]
    tail = text[max(0, n - tail_chars):]
    return head, mid, tail

def _generate_with_mask_from_messages(messages: List[Dict]) -> str:
    """
    attention_mask를 명시해 경고 방지 및 일관 동작 보장.
    """
    inputs = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True, return_tensors="pt", padding=True
    )
    # pad_token_id가 정의되어 있으므로 mask를 정확히 생성
    attention_mask = (inputs != tokenizer.pad_token_id).long()
    with torch.no_grad():
        out = model.generate(
            input_ids=inputs.to(model.device),
            attention_mask=attention_mask.to(model.device),
            **GEN_KW
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

def chat(prompt: str) -> str:
    if tokenizer is None or model is None:
        raise RuntimeError("Model is not loaded.")
    messages = [
        {"role": "system", "content": "당신은 AI 논문/수식을 한국어로 쉽게 설명하는 선생님입니다. 항상 존댓말로 답변합니다."},
        {"role": "user",   "content": prompt}
    ]
    text = _generate_with_mask_from_messages(messages)
    return text.split(messages[-1]["content"])[-1].strip()


# === 셀 5: 수식 해설 생성 ===
EXPLAIN_SYSTEM = "You are a teacher who explains math/AI research equations in clear, simple English. Always answer politely and understandably."
EXPLAIN_TEMPLATE = (
    """Please explain the following equation so that it can be understood by someone at least at a middle school level.
Follow this exact order in your output: Example → Explanation → Conclusion

- Example: Show the equation exactly as LaTeX in a single block (do not modify or add anything).
- Explanation: Provide bullet points explaining the meaning of symbols (∑, 𝟙, ^, _, √, \\, etc.) and the role of each term, in a clear and concise way.
- Conclusion: Summarize in one sentence the core purpose of this equation in the context of the paper (e.g., loss composition, normalization, coordinate error, probability/log-likelihood, etc.).
- (Important) Do not change the symbols or the order of the equation, and do not invent new symbols.
- (Important) Write only in English.

[Equation]
{EQUATION}
"""
)

def explain_equation_with_llm(eq_latex: str) -> str:
    if tokenizer is None or model is None:
        raise RuntimeError("Model is not loaded.")
    messages = [
        {"role": "system", "content": EXPLAIN_SYSTEM},
        {"role": "user",   "content": EXPLAIN_TEMPLATE.format(EQUATION=eq_latex)}
    ]
    text = _generate_with_mask_from_messages(messages)
    return text.split(messages[-1]["content"])[:-1][-1].strip() if messages[-1]["content"] in text else text


# === 셀 6: LaTeX 리포트(.tex) 만들기 ===
def latex_escape_verbatim(s: str) -> str:
    s = s.replace("\\", r"\\")
    s = s.replace("#", r"\#").replace("$", r"\$")
    s = s.replace("%", r"\%").replace("&", r"\&")
    s = s.replace("_", r"\_").replace("{", r"\{").replace("}", r"\}")
    s = s.replace("^", r"\^{}").replace("~", r"\~{}")
    return s

def build_report(overview: str, items: List[Dict]) -> str:
    header = (r"""\\documentclass[11pt]{article}
\\usepackage[margin=1in]{geometry}
\\usepackage{amsmath, amssymb, amsfonts}
\\usepackage{hyperref}
\\usepackage{kotex} % Windows MiKTeX에 설치되어 있어야 함 (없으면 xelatex/xeCJK로 컴파일)
\\setlength{\\parskip}{6pt}
\\setlength{\\parindent}{0pt}
\\title{LaTeX 문서 수식 해설 리포트 (중학생 수준 이상)}
\\author{자동 생성 파이프라인}
\\date{""" + datetime.date.today().isoformat() + r"""}
\\begin{document}
\\maketitle
\\tableofcontents
\\newpage
""")
    parts = [header]
    parts.append(r"\\section*{문서 개요}")
    parts.append(latex_escape_verbatim(overview))
    parts.append("\n\\newpage\n")

    for it in items:
        title = f"라인 {it['line_start']}–{it['line_end']} / {it['kind']} {('['+it['env']+']') if it['env'] else ''}"
        parts.append(f"\\section*{{{latex_escape_verbatim(title)}}}")
        parts.append(it["explanation"])  # LLM 출력 그대로 삽입
        parts.append("\n")

    parts.append("\\end{document}\n")
    return "\n".join(parts)


# === 새로 추가: 수식 개수만 세기 ===
def count_equations_only(input_tex_path: str) -> Dict[str, int]:
    p = Path(input_tex_path)
    if not p.exists():
        raise FileNotFoundError(f"입력 TeX 파일을 찾을 수 없습니다: {input_tex_path}")

    src = p.read_text(encoding="utf-8", errors="ignore")
    offsets = make_line_offsets(src)
    pos_to_line = build_pos_to_line(offsets)

    equations_all = extract_equations(src, pos_to_line)
    equations_advanced = [e for e in equations_all if is_advanced(e["body"])]

    # 콘솔 출력 (즉시 표시)
    print(f"총 수식 개수: {len(equations_all)}", flush=True)
    print(f"중학생 수준 이상으로 분류된 수식: {len(equations_advanced)} / {len(equations_all)}", flush=True)

    return {
        "equations_total": len(equations_all),
        "equations_advanced": len(equations_advanced)
    }


# === 파이프라인 실행 함수 ===
def run_pipeline(input_tex_path: str) -> Dict:
    p = Path(input_tex_path)
    if not p.exists():
        raise FileNotFoundError(f"입력 TeX 파일을 찾을 수 없습니다: {input_tex_path}")

    Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

    src = p.read_text(encoding="utf-8", errors="ignore")

    offsets = make_line_offsets(src)
    pos_to_line = build_pos_to_line(offsets)

    # 1) 수식 추출 & 고난도 분류
    equations_all = extract_equations(src, pos_to_line)
    equations_advanced = [e for e in equations_all if is_advanced(e["body"])]

    # 콘솔 요약 출력 (즉시 표시)
    print(f"총 수식 개수: {len(equations_all)}", flush=True)
    print(f"중학생 수준 이상으로 분류된 수식: {len(equations_advanced)} / {len(equations_all)}", flush=True)

    # 2) 문서 개요 요약
    head, mid, tail = take_slices(src)
    overview_prompt = textwrap.dedent(f"""
    다음 LaTeX 문서의 앞/중/뒤 일부를 보여드립니다.
    - 핵심 주제/목표를 한 문단으로 요약하시고,
    - 주요 섹션(있다면)을 불릿으로 정리해 주세요.
    - 수학 표기/기호 해석에 초점을 맞춰, 전체 흐름을 설명해 주세요.
    - 항상 한국어 존댓말로, 너무 길지 않게.

    [앞부분]
    {head}

    [중간]
    {mid}

    [뒷부분]
    {tail}
    """).strip()
    doc_overview = chat(overview_prompt)

    # 3) 수식별 해설 생성
    explanations: List[Dict] = []
    for idx, item in enumerate(equations_advanced, start=1):
        print(f"[{idx}/{len(equations_advanced)}] 라인 {item['line_start']}–{item['line_end']}", flush=True)
        exp = explain_equation_with_llm(item["body"])
        explanations.append({
            "index": idx,
            "line_start": item["line_start"],
            "line_end": item["line_end"],
            "kind": item["kind"],
            "env": item["env"],
            "equation": item["body"],
            "explanation": exp
        })

    # 4) JSON 저장
    json_path = os.path.join(OUT_DIR, "equations_explained.json")
    Path(json_path).parent.mkdir(parents=True, exist_ok=True)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump({"overview": doc_overview, "items": explanations}, f, ensure_ascii=False, indent=2)
    print(f"저장 완료(JSON): {json_path}", flush=True)

    # 5) LaTeX 리포트(.tex) 생성
    report_tex_path = os.path.join(OUT_DIR, "yolo_math_report.tex")
    report_tex = build_report(doc_overview, explanations)
    Path(report_tex_path).parent.mkdir(parents=True, exist_ok=True)
    Path(report_tex_path).write_text(report_tex, encoding="utf-8")
    print(f"저장 완료(TeX): {report_tex_path}", flush=True)

    return {
        "input": str(p),
        "counts": {
            "equations_total": len(equations_all),
            "equations_advanced": len(equations_advanced)
        },
        "outputs": {
            "json": json_path,
            "report_tex": report_tex_path,
            "out_dir": OUT_DIR
        }
    }


# === FastAPI 앱 정의 ===
app = FastAPI(title="POLO Math Explainer API", version="1.0.0")

class MathRequest(BaseModel):
    path: str

@app.get("/health")
async def health():
    return {
        "status": "ok",
        "python": sys.version.split()[0],
        "torch": torch.__version__,
        "cuda": torch.cuda.is_available(),
        "device": DEVICE,
        "model_loaded": (tokenizer is not None and model is not None)
    }

# 수식 개수만 세는 엔드포인트 (GET/POST)
@app.get("/count/{file_path:path}")
async def count_get(file_path: str):
    try:
        return count_equations_only(file_path)
    except FileNotFoundError as e:
        raise HTTPException(status_code=404, detail=str(e))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"처리 중 오류: {e}")

@app.post("/count")
async def count_post(req: MathRequest):
    try:
        return count_equations_only(req.path)
    except FileNotFoundError as e:
        raise HTTPException(status_code=404, detail=str(e))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"처리 중 오류: {e}")

@app.post("/math")
async def math_post(req: MathRequest):
    try:
        return run_pipeline(req.path)
    except FileNotFoundError as e:
        raise HTTPException(status_code=404, detail=str(e))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"처리 중 오류: {e}")

@app.get("/math/{file_path:path}")
async def math_get(file_path: str):
    try:
        return run_pipeline(file_path)
    except FileNotFoundError as e:
        raise HTTPException(status_code=404, detail=str(e))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"처리 중 오류: {e}")

# 로컬에서 python app.py 로 실행할 때 편의를 위한 엔트리포인트
if __name__ == "__main__":
    import uvicorn
    uvicorn.run("app:app", host="127.0.0.1", port=8000, reload=True)


중학생 수준 이상으로 분류된 수식: 19 / 66


### 셀 4. “문서 전체 이해” 요약(개요/섹션 요지) 생성

> .tex 전체를 그대로 넣기엔 길 수 있으므로 토막 요약 방식(앞/중/뒤 샘플링)으로 개요를 뽑습니다.

In [6]:
# === 셀 4: 문서 전체 개요 생성 (모델 요약) ===
def take_slices(text:str, head_chars=4000, mid_chars=2000, tail_chars=4000):
    n = len(text)
    head = text[:min(head_chars, n)]
    mid_start = max((n//2) - (mid_chars//2), 0)
    mid = text[mid_start: mid_start + min(mid_chars, n)]
    tail = text[max(0, n - tail_chars):]
    return head, mid, tail

head, mid, tail = take_slices(src)

def chat(prompt:str)->str:
    # Qwen2.5 Instruct 포맷 간단 사용
    messages = [
        {"role":"system", "content":"당신은 AI 논문/수식을 한국어로 쉽게 설명하는 선생님입니다. 항상 존댓말로 답변합니다."},
        {"role":"user", "content": prompt}
    ]
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(input_ids=input_ids, **GEN_KW)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # chat 템플릿 접두부 제거
    return text.split(messages[-1]["content"])[-1].strip()

overview_prompt = textwrap.dedent(f"""
다음 LaTeX 문서의 앞/중/뒤 일부를 보여드립니다.
- 핵심 주제/목표를 한 문단으로 요약하시고,
- 주요 섹션(있다면)을 불릿으로 정리해 주세요.
- 수학 표기/기호 해석에 초점을 맞춰, 전체 흐름을 설명해 주세요.
- 항상 한국어 존댓말로, 너무 길지 않게.

[앞부분]
{head}

[중간]
{mid}

[뒷부분]
{tail}
""").strip()

doc_overview = chat(overview_prompt)
print(doc_overview[:1200])


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


assistant
\end{document}


### 셀 5. 수식 해설(예시 → 설명 → 결론) 생성 함수 + 배치 실행

> 각 수식은 원문을 그대로 제시하고, 상징/기호의 의미와 YOLO 맥락(좌표, 지시자 𝟙, √w_i 등)을 반영해 한국어/존댓말/미괄식으로 출력합니다.

In [7]:
# === 셀 5: 수식 해설 생성 ===
EXPLAIN_SYSTEM = "You are a teacher who explains math/AI research equations in clear, simple English. Always answer politely and understandably."

EXPLAIN_TEMPLATE = """Please explain the following equation so that it can be understood by someone at least at a middle school level.
Follow this exact order in your output: Example → Explanation → Conclusion

- Example: Show the equation exactly as LaTeX in a single block (do not modify or add anything).
- Explanation: Provide bullet points explaining the meaning of symbols (∑, 𝟙, ^, _, √, \\, etc.) and the role of each term, in a clear and concise way.
- Conclusion: Summarize in one sentence the core purpose of this equation in the context of the paper (e.g., loss composition, normalization, coordinate error, probability/log-likelihood, etc.).
- (Important) Do not change the symbols or the order of the equation, and do not invent new symbols.
- (Important) Write only in English.

[Equation]
{EQUATION}
"""


def explain_equation_with_llm(eq_latex:str)->str:
    messages = [
        {"role":"system", "content": EXPLAIN_SYSTEM},
        {"role":"user", "content": EXPLAIN_TEMPLATE.format(EQUATION=eq_latex)}
    ]
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(input_ids=input_ids, **GEN_KW)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text.split(messages[-1]["content"])[-1].strip()

# 배치 실행 (너무 많으면 여러 번에 나누세요)
explanations = []
for idx, item in enumerate(equations_advanced, start=1):
    print(f"[{idx}/{len(equations_advanced)}] 라인 {item['line_start']}–{item['line_end']}")
    exp = explain_equation_with_llm(item["body"])
    explanations.append({
        "index": idx,
        "line_start": item["line_start"],
        "line_end": item["line_end"],
        "kind": item["kind"],
        "env": item["env"],
        "equation": item["body"],
        "explanation": exp
    })

# 중간 저장
json_path = os.path.join(OUT_DIR, "equations_explained.json")
with open(json_path, "w", encoding="utf-8") as f:
    json.dump({
        "overview": doc_overview,
        "items": explanations
    }, f, ensure_ascii=False, indent=2)
print(f"저장 완료: {json_path}")


[1/19] 라인 91–91
[2/19] 라인 118–118
[3/19] 라인 120–120
[4/19] 라인 124–124
[5/19] 라인 127–130
[6/19] 라인 138–138
[7/19] 라인 138–138
[8/19] 라인 142–142
[9/19] 라인 150–150
[10/19] 라인 150–150
[11/19] 라인 156–156
[12/19] 라인 156–156
[13/19] 라인 160–160
[14/19] 라인 173–173
[15/19] 라인 173–173
[16/19] 라인 179–185
[17/19] 라인 198–246
[18/19] 라인 248–248
[19/19] 라인 248–248
저장 완료: C:/POLO/polo-system/models/math./_build\equations_explained.json


### 셀 6. LaTeX 리포트 생성(.tex) — 개요 + 수식별 해설

> MiKTeX(Windows) 또는 TeX Live가 있어야 PDF 컴파일 가능합니다. 없으면 다음 셀에서 설치 안내/에러 메시지를 드립니다.

In [8]:
# === 셀 6: LaTeX 리포트(.tex) 만들기 ===
import datetime

report_tex_path = os.path.join(OUT_DIR, "yolo_math_report.tex")

def latex_escape_verbatim(s:str)->str:
    # 설명 텍스트 내 LaTeX 특수문자 간단 이스케이프
    s = s.replace("\\", r"\\")
    s = s.replace("#", r"\#").replace("$", r"\$")
    s = s.replace("%", r"\%").replace("&", r"\&")
    s = s.replace("_", r"\_").replace("{", r"\{").replace("}", r"\}")
    s = s.replace("^", r"\^{}").replace("~", r"\~{}")
    return s

def build_report(overview:str, items:List[Dict])->str:
    header = r"""\documentclass[11pt]{article}
\usepackage[margin=1in]{geometry}
\usepackage{amsmath, amssymb, amsfonts}
\usepackage{hyperref}
\usepackage{kotex} % Windows MiKTeX에 설치되어 있어야 함 (없으면 xelatex/xeCJK로 컴파일)
\setlength{\parskip}{6pt}
\setlength{\parindent}{0pt}
\title{LaTeX 문서 수식 해설 리포트 (중학생 수준 이상)}
\author{자동 생성 파이프라인}
\date{""" + datetime.date.today().isoformat() + r"""}
\begin{document}
\maketitle
\tableofcontents
\newpage
"""
    parts = [header]
    # 개요
    parts.append(r"\section*{문서 개요}")
    parts.append(latex_escape_verbatim(overview))
    parts.append("\n\\newpage\n")

    # 각 수식
    for it in items:
        title = f"라인 {it['line_start']}–{it['line_end']} / {it['kind']} {('['+it['env']+']') if it['env'] else ''}"
        parts.append(f"\\section*{{{latex_escape_verbatim(title)}}}")
        # 예시 → 설명 → 결론 (LLM 출력 그대로 삽입, 단 수식 블록은 유지)
        # 모델 출력 중 코드블록이 있다면 제거하고 LaTeX 수식만 남기는 게 안전
        exp_txt = it["explanation"]
        parts.append(exp_txt)
        parts.append("\n")

    parts.append("\\end{document}\n")
    return "\n".join(parts)

report_tex = build_report(doc_overview, explanations)
Path(report_tex_path).write_text(report_tex, encoding="utf-8")
print(f"LaTeX 리포트 생성: {report_tex_path}")


LaTeX 리포트 생성: C:/POLO/polo-system/models/math./_build\yolo_math_report.tex


딱 여기 셀6까지만 실행하고 파일 확인하시면 됩니다! 7번부터는 이런저런거 깔아야하고 방향성을 다시 잡는 중이라..

section별로 나뉘어 있습니다! 드래그해서 지피티한테 한글로 번역. 틀려도 그대로 이렇게 주문하시면 됩니다!

### 셀 7. PDF 컴파일 시도 (pdflatex → xelatex 순서)

> 미설치 시 MiKTeX 설치 후 PATH 반영이 필요합니다. 실패해도 .tex는 생성되어 있으니, 로컬에서 GUI(MiKTeX Console)로 컴파일하셔도 됩니다.

In [10]:
# === 셀 7: PDF 컴파일 ===
def run_cmd(cmd:List[str])->Tuple[int,str,str]:
    try:
        proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, cwd=OUT_DIR, shell=False)
        return proc.returncode, proc.stdout, proc.stderr
    except FileNotFoundError:
        return 127, "", f"Command not found: {cmd[0]}"

def try_compile(tex_filename:str)->Optional[str]:
    base = os.path.splitext(tex_filename)[0]
    pdf_path = os.path.join(OUT_DIR, base + ".pdf")
    for engine in [["pdflatex","-interaction=nonstopmode","-halt-on-error",tex_filename],
                   ["xelatex","-interaction=nonstopmode","-halt-on-error",tex_filename]]:
        print(">> Trying:", " ".join(engine))
        code,out,err = run_cmd(engine)
        if code == 0 and Path(pdf_path).exists():
            print("PDF 생성 성공:", pdf_path)
            return pdf_path
        else:
            print("실패 코드:", code)
            print(out[:800])
            print(err[:800])
    return None

pdf_path = try_compile("yolo_math_report.tex")
if pdf_path:
    print("완료:", pdf_path)
else:
    print("PDF 컴파일에 실패했습니다. MiKTeX 또는 TeX Live 설치/패키지(kotex) 확인 후 다시 시도해 주세요.")


>> Trying: pdflatex -interaction=nonstopmode -halt-on-error yolo_math_report.tex
실패 코드: 127

Command not found: pdflatex
>> Trying: xelatex -interaction=nonstopmode -halt-on-error yolo_math_report.tex
실패 코드: 127

Command not found: xelatex
PDF 컴파일에 실패했습니다. MiKTeX 또는 TeX Live 설치/패키지(kotex) 확인 후 다시 시도해 주세요.


### 셀 8. 인덱스/로그 저장 (CSV/JSON)

In [ ]:
# === 셀 8: 인덱스/로그 저장 ===
import pandas as pd

df = pd.DataFrame(equations_all)
df_adv = pd.DataFrame(equations_advanced)

df_path = os.path.join(OUT_DIR, "equations_all.csv")
df_adv_path = os.path.join(OUT_DIR, "equations_advanced.csv")
df.to_csv(df_path, index=False, encoding="utf-8-sig")
df_adv.to_csv(df_adv_path, index=False, encoding="utf-8-sig")

print("저장 완료:")
print(" - 전체 수식 CSV :", df_path)
print(" - 고난도 수식 CSV:", df_adv_path)

# 개요/설명 요약본
summary_md = os.path.join(OUT_DIR, "README_report_summary.md")
with open(summary_md, "w", encoding="utf-8") as f:
    f.write("# 문서 개요\n\n")
    f.write(doc_overview + "\n\n")
    f.write("## 수식 통계\n")
    f.write(f"- 전체 수식: {len(equations_all)}개\n")
    f.write(f"- 고난도 수식: {len(equations_advanced)}개\n")
print(" - 요약 MD       :", summary_md)
